In [1]:
import pandas as pd
import numpy as np
import glob

In [ ]:
import xgboost as xgb

# 메모리 사용량 확인 및 정리

In [2]:
def check_memory_usage():
    memory_usage = pd.DataFrame(columns=['Name', 'Memory Usage (MB)'])
    for var_name in dir():
        if var_name.startswith('_') or var_name in ['memory_usage', 'check_memory_usage', 'gc', 'pd', 'np']:
            continue
        var = globals()[var_name]
        if isinstance(var, pd.DataFrame):
            memory_mb = var.memory_usage(deep=True).sum() / 1024 / 1024
            temp_df = pd.DataFrame([[var_name, f"{memory_mb:.2f}"]], columns=['Name', 'Memory Usage (MB)'])
            memory_usage = pd.concat([memory_usage, temp_df], ignore_index=True)

    memory_usage = memory_usage.sort_values(by='Memory Usage (MB)', ascending=False).reset_index(drop=True)
    print(memory_usage.head(10))
    total_memory = float(memory_usage['Memory Usage (MB)'].str.replace(',', '').astype(float).sum())
    print(f"Total Memory Usage: {total_memory:.2f} MB")

# 파일 불러오기

In [ ]:
def load_data(data_type):
    dfs = {}
    base_path = f"/kaggle/input/credit-card-segment/open/{data_type}"
    categories = {
        "customer": "1",
        "credit": "2",
        "sales": "3",
        "billing": "4",
        "balance": "5",
        "channel": "6",
        "marketing": "7",
        "performance": "8"
    }
    for name, prefix in categories.items():
        path =  f"{base_path}/{prefix}.*/*.parquet"
        files = sorted(glob.glob(path))
        if not files:
            print(f"No parquet files found in {path}")
            continue
        df_list = []
        for f in files:
            temp_df = pd.read_parquet(f)

            # 메모리 최적화: 데이터 타입 변환
            for col in temp_df.columns:
                if temp_df[col].dtype == 'float64' and temp_df[col].fillna(0).apply(lambda x: x.is_integer() if isinstance(x, float) else True).all():
                    temp_df[col] = temp_df[col].astype('float32')
                elif temp_df[col].dtype == 'int64':
                    temp_df[col] = temp_df[col].astype('int32')

            df_list.append(temp_df)
            del temp_df
            gc.collect()

        dfs[name] = pd.concat(df_list, ignore_index=True)
        print(f"{name} Loaded {dfs[name].shape} parquet files")

        # 각 카테고리 로드 후 메모리 정리
        del df_list
        gc.collect()
    return dfs

In [ ]:
import gc

In [ ]:
import re

# 전처리 함수들
1. '개, 대, 회, 이상, 회이상, 개이상, 대이상, 대 이상, 회 이상'로 끝나는 문자열에서 숫자만 추출
2. '', 'none', 'null', 'nan', 'NaN' 이 문자열로 들어왔을 때 처리, 10101, -999999, -99, 999, 99999999 값 0 처리
3. unique 값이 1개인 컬럼 drop
4. IQR 방식으로 이상치 탐색 후 NaN 으로 대치 (날짜 형태의 컬럼은 0으로 처리)
5. 날짜형 데이터의 추가적인 결측치가 있을 경우 0처리
6. 각 컬럼별 결측치의 비율에 따라 평균값, 중앙값, 최빈값, drop등으로 처리

## 문자열 -> 수치형으로 파싱

'개, 대, 회, 이상, 회이상, 개이상, 대이상'로 끝나는 문자열에서 숫자만 추출


In [ ]:
def clean_units_from_dataframe(df):
    # 데이터프레임 복사본 생성
    df_cleaned = df.copy()
    object_columns = df_cleaned.select_dtypes(include=['object']).columns
    # 각 object 컬럼 처리
    for col in object_columns:
        if df_cleaned[col].isnull().all():  # 모든 값이 null이면 처리 안함
            continue

        # 문자열인 경우에만 처리
        mask = df_cleaned[col].apply(lambda x: isinstance(x, str))

        if mask.any():  # 문자열이 하나 이상 있는 경우만 처리
            unit_pattern = r'\d+\s*(개|대|회|이상|회이상|개이상|대이상|회 이상|일 이상)$'
            # re.search를 직접 적용하여 경고 피하기
            unit_mask = df_cleaned.loc[mask, col].apply(lambda x: bool(re.search(unit_pattern, x)))

            # 단위가 있는 값만 처리
            if unit_mask.any():
                # 숫자 추출하기 위한 패턴
                number_pattern = r'(\d+)\s*(개|대|회|이상|회이상|개이상|대이상|회 이상|일 이상)$'
                # 숫자 부분만 추출
                extracted_numbers = df_cleaned.loc[mask, col].str.extract(number_pattern)[0]
                # 추출된 숫자를 새로운 값으로 사용 (숫자로 변환)
                df_cleaned.loc[mask & unit_mask, col] = pd.to_numeric(
                    extracted_numbers.loc[unit_mask], errors='coerce')

    # 메모리 최적화
    gc.collect()
    return df_cleaned

## 결측치 수기 처리

'', 'none', 'null', 'nan', 'NaN' 이 문자열로 들어왔을 때 처리, 10101, -999999, -99, 999, 99999999 값 0 처리

In [ ]:
def is_special_null(df):
    # 원본 데이터프레임 수정하기
    for col in df.select_dtypes(include=['object']).columns:
        # 문자열 컬럼에서 빈 문자열, 'none', 'null', 'nan' 찾기
        mask = df[col].astype(str).str.lower().isin(['', 'none', 'null', 'nan', 'NaN'])
        df.loc[mask, col] = np.nan  # NaN으로 변경

    # 숫자 컬럼 처리
    for col in df.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns:
        # 특정 숫자값 찾기
        mask = df[col].isin([10101, -999999, -99, 999, 99999999])
        df.loc[mask, col] = np.nan  # NaN으로 변경

    gc.collect()
    return df

## 컬럼별 데이터 형태가 1개밖에 없는 경우 drop

unique 값이 1개인 컬럼 drop


In [3]:
def remove_single_value_columns(df):
    single_value_cols = []
    for col in df.columns:
        if df[col].nunique() == 1:
            single_value_cols.append(col)

    if single_value_cols:
        print(f"제거할 컬럼 (유니크 값 1개): {single_value_cols}")
        df = df.drop(columns=single_value_cols)
    else:
        print("유니크 값이 1개인 컬럼이 없습니다.")

    gc.collect()
    return df

## 이상치 처리

IQR 방식으로 이상치 탐색 후 NaN 으로 대치 (날짜 형태의 컬럼은 0으로 처리)

날짜형 데이터의 추가적인 결측치가 있을 경우 0처리

In [ ]:
# 이상치 처리 함수 최적화
def handle_outliers_with_iqr(df, date_columns=None):
    if date_columns is None:
        date_columns = []

    # 모든 수치형 컬럼 선택
    numeric_cols = df.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns

    # 각 수치형 컬럼 처리
    for col in numeric_cols:
        # 결측치가 아닌 값으로만 IQR 계산
        valid_data = df[col].dropna()

        if len(valid_data) > 0:  # 유효한 데이터가 있는 경우만 처리
            Q1 = valid_data.quantile(0.25)
            Q3 = valid_data.quantile(0.75)
            IQR = Q3 - Q1

            # 이상치 경계 계산
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # 이상치 찾기
            mask_lower = df[col] < lower_bound
            mask_upper = df[col] > upper_bound

            # 이상치 개수 출력
            outliers_count = mask_lower.sum() + mask_upper.sum()
            if outliers_count > 0:
                print(f"컬럼 '{col}': {outliers_count}개의 이상치 발견")

            # 이상치 처리 - date_columns에 포함된 컬럼이면 0으로, 아니면 NaN으로
            if col in date_columns:
                df.loc[mask_lower | mask_upper, col] = 0
                if outliers_count > 0:
                    print(f"  -> 날짜 컬럼으로 간주하여 이상치를 0으로 대체")
            else:
                df.loc[mask_lower | mask_upper, col] = np.nan
                if outliers_count > 0:
                    print(f"  -> 일반 컬럼으로 간주하여 이상치를 NaN으로 대체")

    gc.collect()
    return df

In [ ]:
def handle_date_columns(df, date_cols):
    for col in date_cols:
        if col in df.columns:
            missing_before = df[col].isnull().sum()
            df[col] = df[col].fillna(0)
            missing_after = df[col].isnull().sum()
            print(f"컬럼 '{col}'의 결측치: {missing_before} -> {missing_after}")

    gc.collect()
    return df

## 결측치 처리

각 컬럼별 결측치의 비율에 따라 평균값, 중앙값, 최빈값, drop등으로 처리

In [ ]:
def handle_missing_values(df):
    # 결측치가 있는 컬럼 찾기
    columns_with_missing = [col for col in df.columns if df[col].isnull().sum() > 0]

    for col in columns_with_missing:
        missing_count = df[col].isnull().sum()
        missing_ratio = missing_count / len(df)
        print(f"컬럼 '{col}'의 결측치: {missing_count}개 ({missing_ratio:.4f})")

        # 결측 비율에 따른 처리
        if missing_ratio > 0.75:
            # 75% 초과 결측: 컬럼 제거
            print(f"  -> 결측치 비율이 75%를 초과합니다. 해당 컬럼 제거")
            df = df.drop(columns=[col])

        elif missing_ratio > 0.5:
            # 50~75% 결측
            if df[col].dtype in ['int32', 'int64', 'float32', 'float64']:
                median_val = df[col].median()
                df[col] = df[col].fillna(median_val)
                print(f"  -> 결측 비율이 높은 수치형 컬럼이므로 중앙값으로 대체")
            else:
                mode_val = df[col].mode()[0] if not df[col].mode().empty else "unknown"
                df[col] = df[col].fillna(mode_val)
                print(f"  -> 결측 비율이 높은 범주형 컬럼이므로 최빈값으로 대체")

        elif missing_ratio > 0.3:
            # 30~50% 결측
            if df[col].dtype in ['int32', 'int64', 'float32', 'float64']:
                try:
                    skew = df[col].skew()
                    if abs(skew) > 1:  # 비대칭 분포인 경우
                        median_val = df[col].median()
                        df[col] = df[col].fillna(median_val)
                        print(f"  -> 분포가 비대칭이므로 중앙값으로 대체")
                    else:  # 대칭 분포인 경우
                        mean_val = df[col].mean()
                        df[col] = df[col].fillna(mean_val)
                        print(f"  -> 분포가 대칭이므로 평균값으로 대체")
                except:
                    # 왜도 계산 오류 시 중앙값 사용
                    median_val = df[col].median()
                    df[col] = df[col].fillna(median_val)
                    print(f"  -> 중앙값으로 대체")
            else:
                mode_val = df[col].mode()[0] if not df[col].mode().empty else "unknown"
                df[col] = df[col].fillna(mode_val)
                print(f"  -> 범주형 컬럼이므로 최빈값으로 대체")

        else:
            # 30% 이하 결측
            if df[col].dtype in ['int32', 'int64', 'float32', 'float64']:
                try:
                    skew = df[col].skew()
                    if abs(skew) > 1:
                        median_val = df[col].median()
                        df[col] = df[col].fillna(median_val)
                        print(f"  -> 분포가 비대칭이므로 중앙값으로 대체")
                    else:
                        mean_val = df[col].mean()
                        df[col] = df[col].fillna(mean_val)
                        print(f"  -> 분포가 대칭이므로 평균값으로 대체")
                except:
                    median_val = df[col].median()
                    df[col] = df[col].fillna(median_val)
                    print(f"  -> 중앙값으로 대체")
            else:
                mode_val = df[col].mode()[0] if not df[col].mode().empty else "unknown"
                df[col] = df[col].fillna(mode_val)
                print(f"  -> 범주형 컬럼이므로 최빈값으로 대체")

    gc.collect()
    return df

# 라벨 인코딩 처리

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
def label_encode_dataframe(df, exclude_cols=None):
    if exclude_cols is None:
        exclude_cols = []

    # object 타입 컬럼 찾기
    object_columns = df.select_dtypes(include=['object']).columns

    # 인코딩할 컬럼 (제외 컬럼 제외)
    encode_columns = [col for col in object_columns if col not in exclude_cols]

    if len(encode_columns) == 0:
        print("인코딩할 object 타입 컬럼이 없습니다.")
        return df

    print(f"총 {len(encode_columns)}개의 object 타입 컬럼에 라벨 인코딩을 적용합니다.")

    # 각 컬럼에 라벨 인코딩 적용
    for col in encode_columns:
        print(f"컬럼 '{col}' 인코딩 중...")

        # 해당 컬럼의 유니크 값 수 확인
        unique_count = df[col].nunique()
        print(f"  -> 유니크 값 수: {unique_count}")

        # 누락된 값이 있는지 확인
        null_count = df[col].isnull().sum()
        if null_count > 0:
            print(f"  -> 주의: {null_count}개의 결측치가 있습니다.")

        # 라벨 인코더 생성 및 적용
        le = LabelEncoder()

        # 결측치가 있는 경우 임시로 처리 (인코딩 후 다시 NaN으로 변경)
        temp_col = df[col].copy()
        mask_null = temp_col.isnull()

        if mask_null.any():
            # 결측치는 임시로 특수 문자열로 대체
            temp_col = temp_col.fillna('NULL_VALUE_FOR_ENCODING')

        # 인코딩 적용
        temp_col = le.fit_transform(temp_col.astype(str))

        # 인코딩된 값 데이터프레임에 대입
        df[col] = temp_col

        # 결측치가 있었다면 다시 NaN으로 변경
        if mask_null.any():
            df.loc[mask_null, col] = np.nan

        print(f"  -> 인코딩 완료: {len(le.classes_)}개의 클래스로 변환")

        # 메모리 정리
        del temp_col, le, mask_null
        gc.collect()

    print("모든 컬럼 인코딩 완료!")
    return df

In [4]:
def process_data():
    print("데이터 로드 시작...")
    train_df = load_data("train")
    test_df = load_data("test")
    print("데이터 로드 완료!")

    # 메모리 정리
    gc.collect()
    check_memory_usage()

    # 데이터 선언
    customer_train_df = train_df["customer"]
    credit_train_df = train_df["credit"]
    sales_train_df = train_df["sales"]
    billing_train_df = train_df["billing"]
    balance_train_df = train_df["balance"]
    channel_train_df = train_df["channel"]
    marketing_train_df = train_df["marketing"]
    performance_train_df = train_df["performance"]

    # 훈련 데이터 사전 정리
    del train_df
    gc.collect()

    # 테스트 데이터 설정
    customer_test_df = test_df["customer"]
    credit_test_df = test_df["credit"]
    sales_test_df = test_df["sales"]
    billing_test_df = test_df["billing"]
    balance_test_df = test_df["balance"]
    channel_test_df = test_df["channel"]
    marketing_test_df = test_df["marketing"]
    performance_test_df = test_df["performance"]

    # 테스트 데이터 사전 정리
    del test_df
    gc.collect()
    check_memory_usage()

    # 각 데이터프레임 처리 - 단위 제거
    print("\n단위 제거 처리 중...")

    # 훈련 데이터 처리
    for df_name in ["customer_train_df", "credit_train_df", "sales_train_df", "billing_train_df",
                    "balance_train_df", "channel_train_df", "marketing_train_df", "performance_train_df"]:
        print(f"{df_name} 단위 제거 중...")
        globals()[df_name] = clean_units_from_dataframe(globals()[df_name])

    # 테스트 데이터 처리
    for df_name in ["customer_test_df", "credit_test_df", "sales_test_df", "billing_test_df",
                    "balance_test_df", "channel_test_df", "marketing_test_df", "performance_test_df"]:
        print(f"{df_name} 단위 제거 중...")
        globals()[df_name] = clean_units_from_dataframe(globals()[df_name])

    gc.collect()
    check_memory_usage()

    # 특수 null 값 처리
    print("\n특수 null 값 처리 중...")

    # 훈련 데이터 처리
    for df_name in ["customer_train_df", "credit_train_df", "sales_train_df", "billing_train_df",
                    "balance_train_df", "channel_train_df", "marketing_train_df", "performance_train_df"]:
        print(f"{df_name} 특수 null 값 처리 중...")
        globals()[df_name] = is_special_null(globals()[df_name])

    # 테스트 데이터 처리
    for df_name in ["customer_test_df", "credit_test_df", "sales_test_df", "billing_test_df",
                    "balance_test_df", "channel_test_df", "marketing_test_df", "performance_test_df"]:
        print(f"{df_name} 특수 null 값 처리 중...")
        globals()[df_name] = is_special_null(globals()[df_name])

    gc.collect()
    check_memory_usage()

    # 단일 값 컬럼 제거
    print("\n단일 값 컬럼 제거 중...")

    # 훈련 데이터 처리
    for df_name in ["customer_train_df", "credit_train_df", "sales_train_df", "billing_train_df",
                    "balance_train_df", "channel_train_df", "marketing_train_df", "performance_train_df"]:
        print(f"{df_name} 단일 값 컬럼 제거 중...")
        globals()[df_name] = remove_single_value_columns(globals()[df_name])

    # 테스트 데이터 처리
    for df_name in ["customer_test_df", "credit_test_df", "sales_test_df", "billing_test_df",
                    "balance_test_df", "channel_test_df", "marketing_test_df", "performance_test_df"]:
        print(f"{df_name} 단일 값 컬럼 제거 중...")
        globals()[df_name] = remove_single_value_columns(globals()[df_name])

    gc.collect()
    check_memory_usage()

    # 날짜 컬럼 정의
    customer_date_columns = ["입회일자_신용", "최종유효년월_신용_이용가능", "최종유효년월_신용_이용", "최종카드발급일자"]
    sales_date_columns = ["최종이용일자_기본", "최종이용일자_신판", "최종이용일자_CA", "최종이용일자_카드론",
                           "최종이용일자_체크", "최종이용일자_일시불", "최종이용일자_할부", "최종카드론_대출일자"]

    # 이상치 처리
    print("\n이상치 처리 중...")

    # 훈련 데이터 이상치 처리
    print("customer_train_df 이상치 처리 중...")
    customer_train_df = handle_outliers_with_iqr(customer_train_df, customer_date_columns)

    for df_name, date_cols in [
        ("credit_train_df", None),
        ("sales_train_df", sales_date_columns),
        ("billing_train_df", None),
        ("balance_train_df", None),
        ("channel_train_df", None),
        ("marketing_train_df", None),
        ("performance_train_df", None)
    ]:
        print(f"{df_name} 이상치 처리 중...")
        globals()[df_name] = handle_outliers_with_iqr(globals()[df_name], date_cols)

    gc.collect()

    # 테스트 데이터 이상치 처리
    print("\n테스트 데이터 이상치 처리 중...")
    print("customer_test_df 이상치 처리 중...")
    customer_test_df = handle_outliers_with_iqr(customer_test_df, customer_date_columns)

    for df_name, date_cols in [
        ("credit_test_df", None),
        ("sales_test_df", sales_date_columns),
        ("billing_test_df", None),
        ("balance_test_df", None),
        ("channel_test_df", None),
        ("marketing_test_df", None),
        ("performance_test_df", None)
    ]:
        print(f"{df_name} 이상치 처리 중...")
        globals()[df_name] = handle_outliers_with_iqr(globals()[df_name], date_cols)

    gc.collect()
    check_memory_usage()

    # 날짜 컬럼 처리
    print("\n날짜 컬럼 처리 중...")
    customer_train_df = handle_date_columns(customer_train_df, customer_date_columns)
    sales_train_df = handle_date_columns(sales_train_df, sales_date_columns)
    customer_test_df = handle_date_columns(customer_test_df, customer_date_columns)
    sales_test_df = handle_date_columns(sales_test_df, sales_date_columns)

    gc.collect()

    # 결측치 처리
    print("\n결측치 처리 중...")

    # 훈련 데이터 결측치 처리
    for df_name in ["customer_train_df", "credit_train_df", "sales_train_df", "billing_train_df",
                    "balance_train_df", "channel_train_df", "marketing_train_df", "performance_train_df"]:
        print(f"{df_name} 결측치 처리 중...")
        globals()[df_name] = handle_missing_values(globals()[df_name])

    # 테스트 데이터 결측치 처리
    for df_name in ["customer_test_df", "credit_test_df", "sales_test_df", "billing_test_df",
                    "balance_test_df", "channel_test_df", "marketing_test_df", "performance_test_df"]:
        print(f"{df_name} 결측치 처리 중...")
        globals()[df_name] = handle_missing_values(globals()[df_name])

    gc.collect()
    check_memory_usage()

    # 라벨 인코딩
    print("\n라벨 인코딩 중...")

    # 훈련 데이터 라벨 인코딩
    exclude_columns = ['ID', 'Segment']
    for df_name in ["customer_train_df", "credit_train_df", "sales_train_df", "billing_train_df",
                    "balance_train_df", "channel_train_df", "marketing_train_df", "performance_train_df"]:
        print(f"{df_name} 라벨 인코딩 중...")
        globals()[df_name] = label_encode_dataframe(globals()[df_name], exclude_columns)

    # 테스트 데이터 라벨 인코딩
    exclude_columns = ['ID']
    for df_name in ["customer_test_df", "credit_test_df", "sales_test_df", "billing_test_df",
                    "balance_test_df", "channel_test_df", "marketing_test_df", "performance_test_df"]:
        print(f"{df_name} 라벨 인코딩 중...")
        globals()[df_name] = label_encode_dataframe(globals()[df_name], exclude_columns)

    gc.collect()
    check_memory_usage()

    # 데이터 병합
    print("\n데이터 병합 중...")

    # 훈련 데이터 병합
    print("훈련 데이터 병합 중...")
    merged_train_df = customer_train_df.copy()

    for df_name in ['credit_train_df', 'sales_train_df', 'billing_train_df', 'balance_train_df',
                    'channel_train_df', 'marketing_train_df', 'performance_train_df']:
        print(f"{df_name} 병합 중...")
        try:
            # '기준년월'과 'ID' 모두를 기준으로 병합
            if '기준년월' in globals()[df_name].columns and '기준년월' in merged_train_df.columns:
                merged_train_df = merged_train_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
            else:
                merged_train_df = merged_train_df.merge(globals()[df_name], on='ID', how='left')
        except Exception as e:
            print(f"병합 에러 발생: {e}")
            print(f"병합 중단: {df_name}")
            continue

        print(f"현재 병합된 데이터프레임 크기: {merged_train_df.shape}")

        # 병합 후 원본 데이터프레임 삭제
        del globals()[df_name]
        gc.collect()

    # 테스트 데이터 병합
    print("\n테스트 데이터 병합 중...")
    merged_test_df = customer_test_df.copy()

    for df_name in ['credit_test_df', 'sales_test_df', 'billing_test_df', 'balance_test_df',
                    'channel_test_df', 'marketing_test_df', 'performance_test_df']:
        print(f"{df_name} 병합 중...")
        try:
            # '기준년월'과 'ID' 모두를 기준으로 병합
            if '기준년월' in globals()[df_name].columns and '기준년월' in merged_test_df.columns:
                merged_test_df = merged_test_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
            else:
                merged_test_df = merged_test_df.merge(globals()[df_name], on='ID', how='left')
        except Exception as e:
            print(f"병합 에러 발생: {e}")
            print(f"병합 중단: {df_name}")
            continue

        print(f"현재 병합된 데이터프레임 크기: {merged_test_df.shape}")

        # 병합 후 원본 데이터프레임 삭제
        del globals()[df_name]
        gc.collect()

    # 고객 데이터프레임 제거
    del customer_train_df, customer_test_df
    gc.collect()
    check_memory_usage()

    # 모델 학습
    print("\nXGBoost 모델 학습 중...")

    # 특성 및 타겟 분리
    print("특성 및 타겟 준비 중...")
    feature_cols = [col for col in merged_train_df.columns if col not in ["ID", "Segment"]]
    X = merged_train_df[feature_cols].copy()
    y = merged_train_df["Segment"].copy()

    # 타겟 인코딩
    le_target = LabelEncoder()
    y_encoded = le_target.fit_transform(y)

    # 메모리에서 merged_train_df 제거
    del merged_train_df
    gc.collect()
    check_memory_usage()

    # 모델 생성 및 학습
    print("XGBoost 모델 학습 시작...")
    model = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,  # 데이터의 80%만 사용하여 메모리 절약 및 과적합 방지
        colsample_bytree=0.8,  # 특성의 80%만 사용하여 메모리 절약 및 과적합 방지
        random_state=42
    )

    # 모델 학습 - 10% 데이터로 먼저 테스트 (메모리 부족 방지)
    print("모델 학습 테스트 중 (10% 데이터)...")
    sample_indices = np.random.choice(len(X), size=int(len(X)*0.1), replace=False)
    X_sample = X.iloc[sample_indices]
    y_sample = y_encoded[sample_indices]

    model.fit(X_sample, y_sample)
    del X_sample, y_sample
    gc.collect()

    # 전체 데이터로 모델 학습 (메모리 문제가 없는 경우)
    try:
        print("전체 데이터로 모델 학습 중...")
        model.fit(X, y_encoded)
    except MemoryError:
        print("메모리 부족으로 샘플링된 데이터로 모델 학습을 진행합니다.")
        sample_size = 0.5  # 50% 데이터만 사용
        sample_indices = np.random.choice(len(X), size=int(len(X)*sample_size), replace=False)
        X_sample = X.iloc[sample_indices]
        y_sample = y_encoded[sample_indices]
        model.fit(X_sample, y_sample)
        del X_sample, y_sample

    # X 제거
    del X, y, y_encoded
    gc.collect()
    check_memory_usage()

    # 예측
    print("\n테스트 데이터 예측 중...")
    X_test = merged_test_df[feature_cols].copy()

    # 결측치 처리 (예측 에러 방지)
    for col in X_test.columns:
        if X_test[col].dtype in ['int32', 'int64', 'float32', 'float64']:
            X_test[col] = X_test[col].fillna(0)
        else:
            X_test[col] = X_test[col].fillna("unknown")

    # ID 컬럼 저장
    test_ids = merged_test_df['ID'].copy()

    # merged_test_df 제거
    del merged_test_df
    gc.collect()

    # 배치 예측 (메모리 부족 방지)
    batch_size = 10000
    total_samples = len(X_test)
    num_batches = (total_samples + batch_size - 1) // batch_size

    all_predictions = []
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, total_samples)

        print(f"배치 {i+1}/{num_batches} 예측 중... (샘플 {start_idx}-{end_idx})")
        batch_pred = model.predict(X_test.iloc[start_idx:end_idx])
        all_predictions.extend(batch_pred)

        gc.collect()

    # 예측 결과를 원래 라벨로 변환
    y_test_pred_labels = le_target.inverse_transform(all_predictions)

    # 결과 데이터프레임 생성
    submission = pd.DataFrame({
        'ID': test_ids,
        'pred_label': y_test_pred_labels
    })

    # ID별로 가장 많이 예측된 세그먼트 선택
    final_submission = submission.groupby("ID")["pred_label"] \
                      .agg(lambda x: x.value_counts().idxmax()) \
                      .reset_index()
    final_submission.columns = ["ID", "Segment"]

    # 결과 저장
    final_submission.to_csv('./submission.csv', index=False)
    print("예측 완료! 제출 파일이 생성되었습니다.")

    return final_submission

In [ ]:
process_data()